# SSMR: generate SSM model from trajectory data

In [1]:
import numpy as np
from copy import deepcopy
import os

In [2]:
%matplotlib qt
import matplotlib.pyplot as plt

In [3]:
%load_ext autoreload
%autoreload 2
import utils as utils
import plot_utils as plot

In [4]:
np.set_printoptions(linewidth=300)

## Settings for SSM model

In [5]:
observables = "delay embedding" # "pos-vel"
TIP_NODE = 51

rDOF = 3
oDOF = 3
SSMDim = 6

## Import and inspect decay data

In [6]:
if observables == "pos-vel":
    output_node = 'all'
elif observables == "delay embedding":
    output_node = TIP_NODE
t_in = 1
t_out = 4

trunk_dir = "../../../soft-robot-control/examples/trunk"
decayData_dir = os.path.join(trunk_dir, "dataCollection/origin")
rest_file = os.path.join(trunk_dir, 'rest_qv.pkl')

Data = {}
Data['oData'] = utils.import_pos_data(decayData_dir, rest_file=rest_file, output_node=output_node, t_in=t_in, t_out=t_out)

nTRAJ = len(Data['oData'])
print("nTRAJ:", nTRAJ)

nTRAJ: 50


## Observables

EITHER compute reduced coordinates using delay embedding on the available oData

In [7]:
if observables == "delay embedding":
    N_DELAY = 4
    Data['yData'] = deepcopy(Data['oData'])
    for i in range(nTRAJ):
        delay_embedded_traj = utils.delayEmbedding(Data['oData'][i][1], up_to_delay=N_DELAY)
        Data['yData'][i][1] = delay_embedded_traj[:, :-4]
        Data['yData'][i][0] = Data['yData'][i][0][4:]

OR use position and velocity of all nodes as observables

In [8]:
if observables == "pos-vel":
    Data['yData'] = deepcopy(Data['oData'])
    for i in range(nTRAJ):
        # observables y are pos and vel of tip
        Data['yData'][i][1] = np.vstack((Data['oData'][i][1][3*TIP_NODE:3*TIP_NODE+3, :], np.gradient(Data['oData'][i][1][3*TIP_NODE:3*TIP_NODE+3, :], 0.01, axis=1)))
        # do SVD on full state + full velocity
        # Data['oData'][i][1] = np.vstack((Data['oData'][i][1], np.gradient(Data['oData'][i][1], axis=1)))
    print(Data['yData'][0][1].shape)

### Plot observables of interest

In [9]:
outdofs = [0, 1, 2]
plt.close('all')
# plot trajectories in 3D [x, y, z] space
plot.traj_3D(Data,
             xyz_idx=[('yData', outdofs[0]), ('yData', outdofs[1]), ('yData', outdofs[2])],
             xyz_names=[r'$x$ [mm]', r'$y$ [mm]', r'$z$ [mm]'])
# plot evolution of x, y and z in time, separately in 3 subplots
highlight_traj = [] # [14, 16, 18, 31, 35, 44] # []
plot.traj_xyz(Data,
             xyz_idx=[('yData', outdofs[0]), ('yData', outdofs[1]), ('yData', outdofs[2])],
             xyz_names=[r'$x$ [mm]', r'$y$ [mm]', r'$z$ [mm]'],
             highlight_idx=highlight_traj)
# plot trajectories in 3D [x, x_dot, z] space / NB: x_dot = v_x
plot.traj_3D(Data,
             xyz_idx=[('yData', outdofs[0]), ('yData', outdofs[0]+oDOF), ('yData', outdofs[2])],
             xyz_names=[r'$x$ [mm]', r'$\dot{x}$ [mm/s]', r'$z$ [mm]'])

Slice trajectories, i.e. remove time before they converge with SSM and cut off the end after complete decay

In [10]:
t_interval = [0.1, 3]
Data['oDataTrunc'] = utils.slice_trajectories(Data['oData'], t_interval)
Data['yDataTrunc'] = utils.slice_trajectories(Data['yData'], t_interval)

## Obtain reduced-order coordinates

### Perform PCA on delay-embedded coordinates to obtain reduced coordinates

In [11]:
# Perform SVD on displacement field
from scipy.sparse.linalg import svds

if observables == "pos-vel":
    svd_data = 'oDataTrunc'
elif observables == "delay embedding":
    svd_data = 'yDataTrunc'

Xsnapshots = np.hstack([yDataTrunc[1] for yDataTrunc in Data[svd_data]])
Xsnapshots = Xsnapshots[:, :]

# print(Xsnapshots.shape)
# print(Xsnapshots[:, 0:10])

show_modes = 9
v, s, u = svds(Xsnapshots, k=max(SSMDim, show_modes), which="LM")
ind = np.argsort(s)[::-1]
s = s[ind]
v = v[:, ind]
# v, s, _ = np.linalg.svd(Xsnapshots, compute_uv=True)
# print(s)
# print(v)
l2vals = s**2

In [12]:
# Plot variance description: we expect three modes to capture almost all variance.
# Note we assume data centered around the origin, which is the fixed point of our system.
plt.close('all')
plot.pca_modes(l2vals, up_to_mode=show_modes)

### Project delay-embedded data onto dominant modes

In [13]:
from copy import deepcopy
Data['etaDataTrunc'] = deepcopy(Data['oDataTrunc'])
if observables == "delay embedding":
    Vde = v[:, :SSMDim]
    for i in range(nTRAJ):
        Data['etaDataTrunc'][i][1] = Vde.T @ Data[svd_data][i][1]
elif observables == "pos-vel":
    assert SSMDim % 2 == 0
    Vde = np.kron(np.eye(2), v[:, :SSMDim//2])
    for i in range(nTRAJ):
        Data['etaDataTrunc'][i][1] = Vde.T @ np.vstack((Data['oDataTrunc'][i][1], np.gradient(Data['oDataTrunc'][i][1], 0.01, axis=1))) # np.vstack((np.gradient(Data['oData'][i][1], axis=1), Data['oData'][i][1])) # Vde.T @ np.vstack((Data[svd_data][i][1], np.gradient(Data[svd_data][i][1], axis=1))) # 

In [14]:
plt.close('all')
# plot first three reduced coordinates
plot.traj_xyz(Data,
              xyz_idx=[('etaDataTrunc', 0), ('etaDataTrunc', 1), ('etaDataTrunc', 2)],
              xyz_names=[r'$x_1$', r'$x_2$', r'$x_3$'])

### Train and test data (train/test split)

In [15]:
indTest = [0, 8, 16, 24]
indTrain = [i for i in range(nTRAJ) if i not in indTest]

## SSMLearn

Start Matlab engine and install/run SSMLearn

In [16]:
import matlab.engine
eng = matlab.engine.start_matlab()
eng.cd("../..", nargout=0)
eng.install(nargout=0)
eng.cd("ROM/")

SSMLearn: Data-driven Reduced Order Models for Nonlinear Dynamical Systems
Maintained by Mattia Cenedese (mattiac@ethz.ch) and Joar Axs (jgoeransson@ethz.ch)
License: GNUv3.0
> In path (line 109)
In addpath (line 86)
In install (line 20)


'/home/jonas/Projects/stanford/SSMR-for-control'

Bring data in a format that is accepted by the (slightly modified) version of SSMLearn (requires a specific cell array structure for input data)

In [17]:
# make data ready for Matlab
yDataTruncTrain_matlab = [[], []]
etaDataTruncTrain_matlab = [[], []]
for i in indTrain:
    yDataTruncTrain_matlab[0].append(matlab.double(initializer=Data['yDataTrunc'][i][0].tolist()))
    yDataTruncTrain_matlab[1].append(matlab.double(initializer=Data['yDataTrunc'][i][1].tolist()))
    etaDataTruncTrain_matlab[0].append(matlab.double(initializer=Data['etaDataTrunc'][i][0].tolist()))
    etaDataTruncTrain_matlab[1].append(matlab.double(initializer=Data['etaDataTrunc'][i][1].tolist()))


### Learn geometry of the SSM

Find parametrization of SSM using SSMLearn

In [18]:
SSMOrder = 3
obsDim = 2*oDOF

IMInfo = eng.IMGeometry(yDataTruncTrain_matlab, SSMDim, SSMOrder,
                        'reducedCoordinates', etaDataTruncTrain_matlab, 'l_vals', 0.)
if observables == "pos-vel":
    IMInfoInv = eng.IMGeometry(etaDataTruncTrain_matlab, obsDim, SSMOrder,
                            'reducedCoordinates', yDataTruncTrain_matlab)
    for key in ['map', 'polynomialOrder', 'dimension', 'nonlinearCoefficients', 'phi', 'exponents', 'H']:
        IMInfo['chart'][key] = IMInfoInv['parametrization'][key]

Data seems to have been fed from Python!
   3



### Learn dynamics on the SSM (reduced dynamics)

Find parametrization of reduced dynamics using SSMLearn

In [19]:
ROMOrder = 3
RDInfo = eng.IMDynamicsFlow(etaDataTruncTrain_matlab, 'R_PolyOrd', ROMOrder, 'style', 'default', 'l_vals', 0.)

Data seems to have been fed from Python!
   3

Estimation of the reduced dynamics... 
 Done. 


Stop Matlab engine -- not needed anymore

In [20]:
# eng.quit()

Convert all matlab double arrays to numpy arrays

In [21]:
def convert_matlab_arrays_to_numpy_arrays(info_dict):
  for k,v in info_dict.items():        
     if isinstance(v, dict):
         convert_matlab_arrays_to_numpy_arrays(v)
     else:            
        if isinstance(v, matlab.double):
           info_dict[k] = np.array(v)

convert_matlab_arrays_to_numpy_arrays(IMInfo)
convert_matlab_arrays_to_numpy_arrays(RDInfo)

Stability analysis of reduced dynamics

In [22]:
assert np.all(np.real(RDInfo['eigenvaluesLinPartFlow']) < 0)
RDInfo['eigenvaluesLinPartFlow']

array([[ -1.57768344+10.78732849j],
       [ -1.68966661+10.92975008j],
       [-27.6672563  +0.j        ],
       [ -8.46099166 +0.j        ],
       [ -1.57768344-10.78732849j],
       [ -1.68966661-10.92975008j]])

## Analyze the obtained mappings of SSM geometry and reduced dynamics

### SSM geometry (parametrization)

In [23]:
yRec = {}
errorGeo = {}
meanErrorGeo = {}

#### Train error

In [24]:
yRec['Train'] = utils.lift_trajectories(IMInfo, [Data['etaDataTrunc'][i] for i in indTrain])
errorGeo['Train'] = utils.compute_trajectory_errors(yRec['Train'], [Data['yDataTrunc'][i] for i in indTrain])[0] * 100

meanErrorGeo['Train'] = np.mean(errorGeo['Train'])
print(f"Average parametrization train error: {meanErrorGeo['Train']:.4e}")

Average parametrization train error: 1.0031e-02


#### Test error

In [25]:
yRec['Test'] = utils.lift_trajectories(IMInfo, [Data['etaDataTrunc'][i] for i in indTest])
errorGeo['Test'] = utils.compute_trajectory_errors(yRec['Test'], [Data['yDataTrunc'][i] for i in indTest])[0] * 100

meanErrorGeo['Test'] = np.mean(errorGeo['Test'])
print(f"Average parametrization test error: {meanErrorGeo['Test']:.4e}")

Average parametrization test error: 1.0893e-02


Plot comparison of SSM-predicted vs. actual test trajectories

In [26]:
plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('yData', 0), ('yData', 1), ('yData', 2)],
                    xyz_names=[r'$x$', r'$y$', r'$z$'],
                    traj_idx=indTest,
                    show=False)
plot.traj_xyz(yRec,
              xyz_idx=[('Test', 0), ('Test', 1), ('Test', 2)],
              xyz_names=[r'$x$', r'$y$', r'$z$'],
              axs=axs, ls=':', color='darkblue')

### Reduced dynamics

In [27]:
etaRec = {}
etaRec['RD'] = utils.advectRD(RDInfo, Data['etaDataTrunc'])[0]
meanErrorDyn = {}

for i in range(nTRAJ):
    if etaRec['RD'][i][1].shape[1] != Data['etaDataTrunc'][i][1].shape[1]:
        print(i)

normedTrajDist = utils.compute_trajectory_errors(etaRec['RD'], Data['etaDataTrunc'])[0]
meanErrorDyn['Train'] = np.mean(normedTrajDist[indTrain]) * 100
meanErrorDyn['Test'] = np.mean(normedTrajDist[indTest]) * 100

print(f"Average dynamics train error: {meanErrorDyn['Train']:.4f}")
print(f"Average dynamics test error: {meanErrorDyn['Test']:.4f}")

/home/jonas/miniconda3/envs/soft/lib/python3.8/site-packages/sklearn/preprocessing/_polynomial.py:440: RuntimeWarning: overflow encountered in multiply
  np.multiply(
/home/jonas/Projects/stanford/SSMR-for-control/ROM/python/utils.py:89: RuntimeWarning: invalid value encountered in matmul
  N = lambda t, y: W_r @ phi(np.atleast_2d(y))
/home/jonas/miniconda3/envs/soft/lib/python3.8/site-packages/sklearn/preprocessing/_polynomial.py:440: RuntimeWarning: overflow encountered in multiply
  np.multiply(
/home/jonas/Projects/stanford/SSMR-for-control/ROM/python/utils.py:89: RuntimeWarning: invalid value encountered in matmul
  N = lambda t, y: W_r @ phi(np.atleast_2d(y))
/home/jonas/miniconda3/envs/soft/lib/python3.8/site-packages/sklearn/preprocessing/_polynomial.py:440: RuntimeWarning: overflow encountered in multiply
  np.multiply(
/home/jonas/Projects/stanford/SSMR-for-control/ROM/python/utils.py:89: RuntimeWarning: invalid value encountered in matmul
  N = lambda t, y: W_r @ phi(np.atle

5
10
12
19
21
22
26
43
47
49
Average dynamics train error: inf
Average dynamics test error: 3.8722


In [28]:
plot_all = False
if plot_all:
    vPlot = range(nTRAJ)
else:
    vPlot = indTest

plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('etaDataTrunc', 0), ('etaDataTrunc', 1), ('etaDataTrunc', 2)],
                    xyz_names=[r'$u_1$', r'$u_2$', r'$u_3$'],
                    traj_idx=vPlot,
                    show=False)
plot.traj_xyz(etaRec,
              xyz_idx=[('RD', 0), ('RD', 1), ('RD', 2)],
              xyz_names=[r'$u_1$', r'$u_2$', r'$u_3$'],
              traj_idx=vPlot,
              axs=axs, ls=':', color='darkblue')


### Global error

In [29]:
errorGlo = {}
meanErrorGlo = {}

#### Train error

In [30]:
yRec['RDTrain'] = utils.lift_trajectories(IMInfo, [etaRec['RD'][i] for i in indTrain])
errorGlo['Train'] = utils.compute_trajectory_errors(yRec['RDTrain'], [Data['yDataTrunc'][i] for i in indTrain])[0] * 100

meanErrorGlo['Train'] = np.mean(errorGlo['Train'])
print(f"Average global train error: {meanErrorGlo['Train']:.4f}")

Average global train error: inf


#### Test error

In [31]:
yRec['RDTest'] = utils.lift_trajectories(IMInfo, [etaRec['RD'][i] for i in indTest])
errorGlo['Test'] = utils.compute_trajectory_errors(yRec['RDTest'], [Data['yDataTrunc'][i] for i in indTest])[0] * 100

meanErrorGlo['Test'] = np.mean(errorGlo['Test'])
print(f"Average global test error: {meanErrorGlo['Test']:.4f}")

Average global test error: 3.8733


In [32]:
plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('yData', 0), ('yData', 1), ('yData', 2)],
                    xyz_names=[r'$x$', r'$y$', r'$z$'],
                    traj_idx=indTest,
                    show=False)
plot.traj_xyz(yRec,
              xyz_idx=[('RDTest', 0), ('RDTest', 1), ('RDTest', 2)],
              xyz_names=[r'$x$', r'$y$', r'$z$'],
              axs=axs, ls=':', color='darkblue')

### Understand which mode is responsible for what motion

In [33]:
if observables == "pos-vel":
    nDOF = 3 * 709
    outdofsMat = np.zeros((3, 2*nDOF))
    outdofsPS = 3 * TIP_NODE + np.array([0, 1, 2])
    for i in range(3):
        outdofsMat[i, outdofsPS[i]] = 1

    rDOF = SSMDim // 2
    redDynLinearPart = np.array(RDInfo['reducedDynamics']['coefficients'])[:, :2*rDOF]
    redStiffnessMat = -redDynLinearPart[rDOF:, :rDOF]
    print(redStiffnessMat.shape)

    w2, UconsRedDyn = np.linalg.eig(redStiffnessMat)
    sorted_idx = np.argsort(w2)
    consFreqs, UconsRedDyn = np.diag(np.sqrt(w2[sorted_idx])), UconsRedDyn[:, sorted_idx]

    Ucons = np.kron(np.eye(2), UconsRedDyn)
    Vcons = Vde @ Ucons

    modesDir = outdofsMat @ Vcons[:, :rDOF]
    modesFreq = np.abs(RDInfo['eigenvaluesLinPartFlow']).flatten()[:3]

    plt.close('all')
    plot.mode_direction(modesDir, modesFreq)

## Control

### Setup model for control

In [34]:
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d 

In [35]:
Rauton = utils.Rauton(RDInfo)
Vauton = lambda x: IMInfo['parametrization']['H'] @ utils.multivariate_polynomial(x, IMInfo['parametrization']['polynomialOrder'])
if observables == "pos-vel":
    Wauton = lambda y: IMInfo['chart']['H'] @ utils.multivariate_polynomial(y, IMInfo['chart']['polynomialOrder'])
elif observables == "delay embedding":
    Wauton = lambda y: Vde.T @ y

### Learn control matrix $B$

Use a closed-loop figure8 from a previous test run as the training data

In [36]:
# inputData_dir = os.path.join(trunk_dir, "dataCollection/closed-loop")
# (t_train, z_shift_train), u_train = utils.import_pos_data(inputData_dir, rest_file, TIP_NODE, return_inputs=True)
# print(z_shift_train.shape)
# dt = 0.01

Use randomly sampled inputs as the training and test data

In [85]:
inputData_dir = os.path.join(trunk_dir, "dataCollection/open-loop")
(t, z_shift), u = utils.import_pos_data(inputData_dir, rest_file, TIP_NODE, return_inputs=True)
print(z_shift.shape)
dt = 0.01

(3, 25001)


Train/test split

In [86]:
train_ratio = 0.8
split_idx = int(train_ratio * len(t))
t_train, t_test = t[:split_idx], t[split_idx:]
z_shift_train, z_shift_test = z_shift[:, :split_idx], z_shift[:, split_idx:]
u_train, u_test = u[:, :split_idx], u[:, split_idx:]

Plot the training data

In [87]:
plt.close('all')
plot.traj_xyz_txyz(t=t_train,
                   x=z_shift_train[0, :], y=z_shift_train[1, :], z=z_shift_train[2, :])
plot.inputs(t_train, u_train)

Plot the test data

In [88]:
plt.close('all')
plot.traj_xyz_txyz(t=t_test,
                   x=z_shift_test[0, :], y=z_shift_test[1, :], z=z_shift_test[2, :])
plot.inputs(t_test, u_test)

Do delay embedding on the tip position. Then, compute reduced coordinates and gradient as predicted by reduced dynamics, as well as the numerical (true) gradient

In [89]:
if observables == "delay embedding":
    # delay-embed tip position
    y_train = utils.delayEmbedding(z_shift_train, embed_coords=[0, 1, 2], up_to_delay=N_DELAY)
elif observables == "pos-vel":
    y_train = np.vstack((z_shift_train, np.gradient(z_shift_train, dt, axis=1)))

print(Vde.shape)
print(f"y.shape: {y_train.shape}, u.shape: {u_train.shape}")
# reduced coordinates
x_train = Wauton(y_train)
dxDt = np.gradient(x_train, dt, axis=1)
dxDt_ROM = Rauton(x_train)

# optionally, throw away some of the training data
keep_indices = np.s_[10:-10]
t_train_trunc = t_train[keep_indices]
u_train_trunc = u_train[:, keep_indices]
x_train_trunc = x_train[:, keep_indices]
y_train_trunc = y_train[:, keep_indices]
dxDt = dxDt[:, keep_indices]
dxDt_ROM = dxDt_ROM[:, keep_indices]

print(t_train_trunc.shape)

(15, 6)
y.shape: (15, 20000), u.shape: (8, 20000)
(19980,)


In [90]:
plt.close('all')
ax = plot.traj_3D_xyz(y_train_trunc[outdofs[0], :],
                      y_train_trunc[outdofs[1], :],
                      y_train_trunc[outdofs[2], :], show=True)

Plot gradients (numerical differentiation and reduced dynamics)

In [91]:
print(dxDt.shape)
plot.reduced_coordinates_gradient(t_train_trunc, [dxDt, dxDt_ROM], labels=["true numerical", "predicted autonomous"], how="all")

(6, 19980)


In [92]:
# plt.close('all')
# plot.dependence_of_xdot_on_inputs(dxDt, u_train_trunc)

Learn $B$ matrix

In [93]:
from sklearn.linear_model import Ridge

polyUorder = 2
# assemble_features = lambda u, x: utils.multivariate_polynomial(np.vstack([u, x]), order=polyUorder)
assemble_features = lambda u, x: utils.multivariate_polynomial(u, order=polyUorder)

X = assemble_features(u_train_trunc, x_train_trunc)
print(X.shape)

ridgeModel = Ridge(alpha=0, fit_intercept=False)
ridgeModel.fit(X.T, (dxDt - dxDt_ROM).T)
B_learn = ridgeModel.coef_
print(B_learn.shape)

(44, 19980)
(6, 44)


Frobenius norm of $B_{learn}$ to estimate how much influence inputs have on dynamics

In [94]:
print(f"Frobenius norm of B_learn: {np.linalg.norm(B_learn, ord='fro'):.4f}")

Frobenius norm of B_learn: 1.0037


Plot gradients again, but this time, using the learned influence of inputs for the prediction

In [95]:
plt.close('all')
plot_reduced_coords = np.s_[:] # [3, 4, 5]
dxDt_ROM_with_B = Rauton(x_train_trunc) + B_learn @ assemble_features(u_train_trunc, x_train_trunc) # utils.multivariate_polynomial(u_train_trunc, order=polyUorder)
plot.reduced_coordinates_gradient(t_train_trunc, [dxDt[plot_reduced_coords, :], dxDt_ROM[plot_reduced_coords, :], dxDt_ROM_with_B[plot_reduced_coords, :]], labels=["true numerical", "predicted autonomous", "predicted with inputs"], how="all")

In [96]:
# # home-brewed ridge regression
# alpha = 0
# X, y = U.T, (dXbarDt - dXbarDt_ROM).T
# B_learn = np.linalg.solve(X.T @ X + alpha * np.eye(X.shape[0]), X.T @ y)

### Integrate model with inputs using learned influence of control (open-loop prediction)

Try to predict the test trajectory using reduced dynamics

In [97]:
if observables == "delay embedding":
    # delay-embed tip position
    y_test = utils.delayEmbedding(z_shift_test, embed_coords=[0, 1, 2], up_to_delay=N_DELAY)
elif observables == "pos-vel":
    y_test = np.vstack((z_shift_test, np.gradient(z_shift_test, dt, axis=1)))

# reduced coordinates
x_test = Wauton(y_test)

slice_traj = np.s_[:]
t_test = t_test[slice_traj]
u_test = u_test[:, slice_traj]
x_test = x_test[:, slice_traj]
z_shift_test = z_shift_test[:, slice_traj]

In [98]:
plt.close('all')
ax = plot.traj_3D_xyz(y_test[outdofs[0], :],
                      y_test[outdofs[1], :],
                      y_test[outdofs[2], :], show=True)

In [99]:
uInterpFun = interp1d(t_test, u_test, axis=1, fill_value="extrapolate")
uFun = lambda t: uInterpFun(t).reshape(-1, 1)
# print(t_test.shape, dxDt.shape)
# dxDtInterpFun = interp1d(t_test, dxDt_ROM, axis=1, fill_value="extrapolate")
def R(t, y):
    return Rauton(np.atleast_2d(y)) + B_learn @ assemble_features(uFun(t), y) # utils.multivariate_polynomial(uFun(t), order=polyUorder)
    # return dxDtInterpFun(t)

# solve IVP of reduced dynamics using open-loop figure8 inputs
sol = solve_ivp(R,
                t_span=[t_test[0], t_test[-1]],
                t_eval=t_test,
                y0=x_test[:, 0], # np.zeros(2*rDOF),
                method='RK45',
                vectorized=True,
                rtol=1e-3,
                atol=1e-3)
# resulting (predicted) open-loop trajectory in reduced coordinates
xTraj = sol.y
yTraj = Vauton(xTraj)
zTraj = yTraj[:3, :]
print(zTraj.shape)
if zTraj.shape[1] < len(t_test):
    zTraj = np.hstack((zTraj, np.tile(np.nan, (3, len(t_test) - zTraj.shape[1]))))


(3, 5001)


In [100]:
plt.close('all')
axs = plot.traj_xyz_txyz(   t_test,
                            zTraj[outdofs[0], :],
                            zTraj[outdofs[1], :],
                            zTraj[outdofs[2], :], show=False)
axs = plot.traj_xyz_txyz(   t_test,
                            z_shift_test[outdofs[0], :],
                            z_shift_test[outdofs[1], :],
                            z_shift_test[outdofs[2], :], color="tab:orange", axs=axs, show=False)
axs[-1].legend(["Predicted trajectory", "Actual trajectory"])
plt.show()

In [101]:
RMSE = np.sum(np.sqrt(np.mean((zTraj[0:3, :] - z_shift_test[-3:])**2, axis=0))) / zTraj.shape[1]
print(f"RMSE = {RMSE:.4f}")

RMSE = 0.3846


## Save SSM model

Continuous model

In [102]:
SSM_model = {'model': {}, 'params': {}}

SSM_model['model']['w_coeff'] = IMInfo['parametrization']['H']
SSM_model['model']['r_coeff'] = RDInfo['reducedDynamics']['coefficients']
SSM_model['model']['B'] = B_learn

SSM_model['params']['SSM_order'] = SSMOrder
SSM_model['params']['ROM_order'] = ROMOrder
SSM_model['params']['state_dim'] = SSMDim
SSM_model['params']['u_order'] = polyUorder
SSM_model['params']['input_dim'] = 8

if observables == "delay embedding":
    SSM_model['model']['V'] = Vde
    SSM_model['model']['v_coeff'] = None
    SSM_model['params']['delays'] = N_DELAY
    SSM_model['params']['delay_embedding'] = True
    SSM_model['params']['obs_dim'] = 3 * (1 + N_DELAY)
    SSM_model['params']['output_dim'] = oDOF

elif observables == "pos-vel":
    SSM_model['model']['V'] = None
    SSM_model['model']['v_coeff'] = IMInfo['chart']['H']
    SSM_model['params']['delay_embedding'] = False
    SSM_model['params']['delays'] = 0
    SSM_model['params']['obs_dim'] = 6
    SSM_model['params']['output_dim'] = 2 * oDOF


In [103]:
import pickle
# save dictionary to person_data.pkl file
if observables == "pos-vel":
    model_name = "SSM_model.pkl"
elif observables == "delay embedding":
    model_name = "SSM_model.pkl"

with open(f'../../../soft-robot-control/examples/trunk/SSMmodels/{model_name}', 'wb') as f:
    pickle.dump(SSM_model, f)